In [3]:
# Imports
%%capture

!pip install datasets # !pip install datasets==1.0.2
!pip install transformers # !pip install transformers==4.0.1

import datasets
import pandas as pd

from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import EncoderDecoderModel

# !rm -rf "./checkpoint-100"

In [4]:
# Load data
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:2%]")

train_data.info.description
df = pd.DataFrame(train_data[:1])
del df["id"]

for column, typ in train_data.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])

display(HTML(df.to_html())) # TODO: Rename (text = article, summary = highlights)

In [5]:
# Load tokenizer
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") # TODO: Change tokenizer to a longformer

In [6]:
# Analyze data
def map_to_length(x):
    x["article_len"] = len(tokenizer(x["article"]).input_ids)
    x["article_longer_512"] = int(x["article_len"] > 512) # TODO: Set tokenizer.max_len
    x["summary_len"] = len(tokenizer(x["highlights"]).input_ids)
    x["summary_longer_64"] = int(x["summary_len"] > 64)
    x["summary_longer_128"] = int(x["summary_len"] > 128)
    
    return x
    
sample_size = 10000
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)

def compute_and_print_stats(x):
    if len(x["article_len"]) == sample_size:
        print("Article Mean: {}\n%-Articles > 512: {}\nSummary Mean: {}\n%-Summary > 64: {}\n%-Summary > 128: {}\n".format(
                sum(x["article_len"]) / sample_size,
                sum(x["article_longer_512"]) / sample_size, 
                sum(x["summary_len"]) / sample_size,
                sum(x["summary_longer_64"]) / sample_size,
                sum(x["summary_longer_128"]) / sample_size)
        )

output = data_stats.map(
    compute_and_print_stats, 
    batched=True,
    batch_size=-1,
)

Article Mean: 847.6216
%-Articles > 512: 0.7355
Summary Mean: 57.7742
%-Summary > 64: 0.3185
%-Summary > 128: 0.0




In [7]:
# Prepare data
encoder_max_length=512
decoder_max_length=128
batch_size = 4 # 16

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

train_data = train_data.select(range(32))
val_data = val_data.select(range(8))

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

train_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"],
)

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

val_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"],
)

In [8]:
# Load models
tf2tf = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased") # TODO: Change encoder and decoder, set tie_encoder_decoder=True
tf2tf.save_pretrained("bert2bert")
tf2tf = EncoderDecoderModel.from_pretrained("bert2bert")
tf2tf.config

print(f"Parameters: {tf2tf.num_parameters()}")

Parameters: 247363386


In [9]:
# Configure models
tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
tf2tf.config.eos_token_id = tokenizer.sep_token_id
tf2tf.config.pad_token_id = tokenizer.pad_token_id
tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size

In [10]:
# Configure beam search
tf2tf.config.max_length = 142
tf2tf.config.min_length = 56
tf2tf.config.no_repeat_ngram_size = 3
tf2tf.config.early_stopping = True
tf2tf.config.length_penalty = 2.0
tf2tf.config.num_beams = 4

In [11]:
# Imports
%%capture

!rm seq2seq_trainer.py
!rm seq2seq_training_args.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_training_args.py

!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu

from seq2seq_trainer import Seq2SeqTrainer
from seq2seq_training_args import Seq2SeqTrainingArguments

In [12]:
# Prepare metric
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# Setup trainer
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    warmup_steps=2000,
    num_train_epochs=20000,
    save_steps=5000,
    logging_steps=5000,
    eval_steps=2000,
    save_total_limit=50
)

# TODO: Set more parameters
# tf2tf = EncoderDecoderModel.from_pretrained("./checkpoint-200")

trainer = Seq2SeqTrainer(
    model=tf2tf,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()
!ls

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Runtime,Samples Per Second
2000,0.014600,10.766622,0.000000,0.000000,0.000000,4.395100,1.820000
4000,0.002800,11.471950,0.000000,0.000000,0.000000,4.633700,1.726000
6000,0.005200,11.806880,0.000000,0.000000,0.000000,4.420700,1.810000
8000,0.003500,11.636637,0.002500,0.003900,0.003000,4.557700,1.755000


In [14]:
# Evaluate training
def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred_summary"] = output_str

    return batch

results = test_data.map(
    generate_summary,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article"]
)

# TODO: Show example-result

rouge.compute(predictions=results["pred_summary"], references=results["highlights"], rouge_types=["rouge2"])["rouge2"].mid

# from transformers import BertTokenizer
# bert2bert = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail").to("cuda")
# tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

Score(precision=0.0031365164814984754, recall=0.005147076505225773, fmeasure=0.003777704140790065)